## Database Backend - Influx

### Imports

In [ ]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
import sys, os
import pandas as pd
from datetime import date, datetime, timedelta
import pytz
import altair as alt


## Set influx connection

In [ ]:
# List installed package versions
packList = ["pandas", "altair", "numpy", "influxdb_client"]
for p in packList:
    try:
        mod = __import__(p)
        print(mod.__name__ + " : " + mod.__version__)
    except ImportError:
        print(p + " not installed")

# Get the current working directory
thisDir = os.getcwd()

# List files in the parent directory
parentDir = os.path.abspath(os.path.join(thisDir, os.pardir))
files_in_parent = os.listdir(parentDir)
print("Files in parent directory:", files_in_parent)

## Read in credentials

Sets a dictionay with:
- 'bucket': "BUCKET_NAME",
- 'org': "ORGANISATION_NAME",
- 'token': "TOKEN",
- 'url': "IP:PORT"


In [ ]:
### set up Kenny's credentials (ignore if user!=Kenny)
credDir=os.getcwd()
if os.path.isdir(credDir):
    print("directory found:",credDir)
    sys.path.insert(1, credDir)
    import connectionDetails
    credDict=connectionDetails.GetInfluxCredentials()
    print(credDict)
    print("done.")
else:
    print("no directory found:",credDir)

## Query Influx

### get API client

In [ ]:
### set client to get access to influx
clientV2_remote = influxdb_client.InfluxDBClient(
   url=credDict['url'],
   token=credDict['token'],
   org=credDict['org']
)

### check buckets

In [ ]:
### set bucket api
buckets_api_remote = clientV2_remote.buckets_api()

In [ ]:
### Access local or VPN connected ports
### list buckets (by name)
try:
    #print([x.name for x in buckets_api_remote.find_buckets().buckets])
    database_list=[x.name for x in clientV2_remote.buckets_api().find_buckets().buckets]
    print(database_list)
except:
    print("cannot get buckets")

### Query data

Example query from influxdb GUI (replace all-caps):
<code>
from(bucket: BUCKET_NAME)
  |> range(start: v.timeRangeStart, stop: v.timeRangeStop)
  |> filter(fn: (r) => r["_measurement"] == DATA_NAME)
  |> aggregateWindow(every: 1m, fn: mean, createEmpty: false)
  |> yield(name: "mean")
</code>

In [ ]:
### set query api
query_api_remote = clientV2_remote.query_api()

In [ ]:
### build query string
bucketName="REMS"
start_time_str="2023-10-01_13:30:00" # format: '%Y-%m-%d_%H:%M:%S'
# end_time_str="2023-10-02_13:30:00" # format: '%Y-%m-%d_%H:%M:%S'
# set time period, e.g. start time + 1 hr
local = pytz.timezone("Europe/London")
start_time=datetime.strptime(str(start_time_str), '%Y-%m-%d_%H:%M:%S')
start_local = local.localize(start_time, is_dst=None)
start_utc = start_local.astimezone(pytz.utc)
offset_time=3600 #s
end_time=start_time + timedelta(seconds=offset_time)
end_local = local.localize(end_time, is_dst=None)
end_utc = end_local.astimezone(pytz.utc)
# set filters
filters={'_measurement':"data",
         '_field':"humidity",
         'location':"lab_gladd1_env",
         'sensor':"SHT85"
         }
# build string
build_query_str = ' from(bucket: \"'+bucketName+'\") |> range(start: '+start_utc.strftime('%Y-%m-%dT%H:%M:%SZ')+', stop: '+end_utc.strftime('%Y-%m-%dT%H:%M:%SZ')+')'
for k,v in filters.items():
    build_query_str+=' |> filter(fn: (r) => r["'+k+'"] == "'+v+'")'
build_query_str+=' |> yield(name: "mean")'
print(build_query_str)

In [ ]:
### get data (into dataframe)
build_result = query_api_remote.query_data_frame(org=credDict['org'], query=build_query_str)
display(build_result)

In [ ]:
### copy grafana query
grafana_copy_str='from(bucket: "REMS") \
  |> range(start: 2023-10-01T21:52:26.4Z, stop: 2023-10-02T21:52:26.4Z) \
  |> filter(fn: (r) => r["_measurement"] == "data") \
  |> filter(fn: (r) => r["_field"] == "humidity") \
  |> filter(fn: (r) => r["location"] == "lab_gladd1_env") \
  |> filter(fn: (r) => r["sensor"] == "SHT85") \
  |> filter(fn: (r) => r._value > -900 ) \
  |> aggregateWindow(every: 1m0s, fn: mean, createEmpty: false) \
  |> map(fn: (r) => ({  r with location: "Humidity" })) \
  |> yield(name: "mean")'
print(grafana_copy_str)

In [ ]:
### get data (into dataframe)
copy_result = query_api_remote.query_data_frame(org=credDict['org'], query=grafana_copy_str)
display(copy_result)

## Visualisation
Using altair

In [ ]:
### plot data from build query
alt.Chart(build_result).mark_line(point=True).encode(
    x=alt.X('_time'),
    y=alt.Y('_value'),
    tooltip=['_time','_value']
).properties(title="results of custom made query")


In [ ]:
### plot data from grafana query
alt.Chart(copy_result).mark_line(point=True).encode(
    x=alt.X('_time'),
    y=alt.Y('_value'),
    tooltip=['_time','_value']
).properties(title="results of grafana-like query")
